# Base definition

In [ ]:
class NotValidWaveException(Exception):
    def __init__(self, Rule):
        self.Rule = Rule

    def __str__(self):
        return self.Rule.desp


    
class Point():
    def __init__(self, time_offset: int, price: float):
        self.time_offset = time_offset
        self.price = price
        
class Wave():
    rule_list: list = [] #该类浪需要满足的规则列表
    guide_dict: dict = {} #该浪可以进行加权的指南以及权重
    min_point_num = 0
    max_point_num = 999
    
    def __init__(self, point_list: list):
        # 子浪序列，初始为 None
        self.sub_wave: list[Wave] = []
        self.point_list = point_list
        if len(point_list) < self.min_point_num:
            raise NotValidWaveException(PointNumberRule)
        
    def generate(start_point: Point, end_point: Point): #生成一个浪
        pass
    
    def is_valid(self):
        """
        看是否符合该浪的定义
        """
        for rule in self.rule_list:
            result = rule.validate(self)
            if not result:
                return False
        return True
    
    def get_not_valid_rule(self):
        result = []
        for rule in self.rule_list:
            if not rule.validate(self):
                result.append(rule)
        return result
    
    def set_sub_wave(self, sub_wave_num, sub_wave):
        self.sub_wave[sub_wave_num] = sub_wave
            
    def get_score(self):
        """
        看当前的浪分布满足多少指南
        """
        total_score = 0
        for guide, weight in self.guide_dict.items():
            total_score += weight * guide.get_score(self)
        return total_score
    
    def get_score_contribution(self):
        """
        看当前的浪分布满足指南的列表
        """
        score_reason = {}
        for guide, weight in self.guide_dict.items():
            score_reason[guide.desp] = weight * guide.get_score(self)
        return score_reason
    
    def get_sub_wave_move(self, sub_wave_num):
        return abs(self.point_list[sub_wave_num+1].price - self.point_list[sub_wave_num].price) / self.point_list[sub_wave_num].price

class Rule():
    desp = ""
    def validate(wave: Wave):
        raise NotImplementedError
        

class Guide():
    desp = ""
    def get_score(wave:Wave):
        pass

In [ ]:
class PointNumberRule(Rule):
    desp = "不满足初始点个数要求"

class SubWaveRule():
    desp = ""
    @staticmethod
    def skip_empty_subwave_with_num(skip_sub_wave_num):
        def skip_empty_subwave(func):
            def validate_wrapper(wave: Wave):
                if len(wave.sub_wave) == 0:
                    return True
                # If not set, test all subwave
                if skip_sub_wave_num is None:
                    for this_sub_wave in wave.sub_wave:
                        if this_sub_wave is None:
                            return True
                else:
                    # If sub_wave_num is set, test only this subwave
                    if wave.sub_wave[skip_sub_wave_num] is None:
                        return True
                return func(wave)
            return validate_wrapper
        return skip_empty_subwave

class Rule1(SubWaveRule):
    desp = "浪1总是一个推动浪或者斜纹浪"    
    @SubWaveRule.skip_empty_subwave_with_num(0)
    def validate(wave: Wave):
        return isinstance(wave.sub_wave[0], MotiveWave)

class Rule2(Rule):
    desp = "浪2永远不会超过浪1的起点"
    def validate(wave: Wave):
        return wave.point_list[2].price >= wave.point_list[0].price

class Rule3(SubWaveRule):
    desp = "浪2总是细分成一个锯齿形调整浪 Or 平台型调整浪 Or 联合型调整浪"
    @SubWaveRule.skip_empty_subwave_with_num(1)
    def validate(wave: Wave):
        return isinstance(wave.sub_wave[1], ZigZagWave) or isinstance(wave[1].sub_wave[1], FlatWave) or isinstance(wave[1].sub_wave[1], CombinationWave) 

class Rule4(Rule):
    desp = "浪3永远不是最短的一浪"
    def validate(wave: Wave):
        wave_start_point = wave.point_list[:-1]
        wave_end_point = wave.point_list[1:]
        time_period_list = [ a.time_offset - b.time_offset for a, b in zip(wave_end_point, wave_start_point)]
        min_time = min(time_period_list)
        return time_period_list[2] >= min_time
                
class Rule5(SubWaveRule):
    desp = "浪3总是一个推动浪"
    @SubWaveRule.skip_empty_subwave_with_num(2)
    def validate(wave: Wave):
        return isinstance(wave.sub_wave[2], MotiveWave)

class Rule6(Rule):
    desp = "浪3总是运动过浪1的终点"
    def validate(wave: Wave):
        return wave.point_list[3].price >= wave.point_list[1].price

class Rule7(Rule):
    desp = "浪4永远不会进入浪1的价格区域"
    def validate(wave: Wave):
        start_price = wave.point_list[3].price
        end_price = wave.point_list[4].price
        
        return start_price > wave.point_list[1].price and end_price > wave.point_list[1].price
        # TODO: test all points in subwave too

class Rule8(SubWaveRule):
    desp = "浪4总是细分成一个锯齿形调整浪 Or 平台型调整浪Or 三角形调整浪 Or 联合型调整浪"
    @SubWaveRule.skip_empty_subwave_with_num(3)
    def validate(wave: Wave):
        return isinstance(wave.sub_wave[3], CorrectiveWave)

class Rule9(SubWaveRule):
    desp = "浪5总是一个推动浪或者斜纹浪"
    @SubWaveRule.skip_empty_subwave_with_num(4)
    def validate(wave: Wave):
        return isinstance(wave.sub_wave[4], MotiveWave)
    
class Rule10(SubWaveRule):
    desp = "浪1、3、5最多只有两个延长浪，不会3个都延长"
    @SubWaveRule.skip_empty_subwave_with_num
    def validate(wave: Wave):
        extend_num = 0
        for wave_num in [0,2,4]:
            if (wave.sub_wave[wave_num].is_extend_wave):
                extend_num += 1
        return extend_num < 3

In [ ]:
def wave_move_is_fibonacci(wave, wave_num_a, wave_num_b):
    """
    浪a 和 浪b 的净位移是斐波那契比率
    """
    move1 = wave.get_sub_wave_move(wave_num_a)
    move2 = wave.get_sub_wave_move(wave_num_b)
    if abs(move2/move1 - 0.618) < 0.01:
        return 1
    if abs(move1/move2 - 0.618) < 0.01:
        return 1
    return 0

def sub_wave_is_extend(wave, sub_wave_num):
    sub_wave = wave.sub_wave[sub_wave_num]
    if sub_wave and sub_wave.is_extend_wave:
        return True
    return False



In [ ]:
class ImpluseWaveGuide1(Guide):
    desp = "其中一个驱动浪常常会延长，导致出现9浪"
    def get_score(wave:Wave):
        for i in range(5):
            if sub_wave_is_extend(wave, i):
                return 1
        return 0
    
class ImpluseWaveGuide2(Guide):
    desp = "浪3延长比较常见"
    def get_score(wave:Wave):
        if sub_wave_is_extend(wave, 2):
            return 1
        return 0
    
class ImpluseWaveGuide3(Guide):
    desp = "两个浪都延长很罕见"
    def get_score(wave:Wave):
        extend_wave = 0
        for sub_wave in wave.sub_wave:
            if sub_wave is not None and sub_wave.is_extend_wave:
                extend_wave += 1
        if extend_wave == 2:
            return -1
        return 0

class ImpluseWaveGuide4(Guide):
    desp = "浪1最不可能延长"
    def get_score(wave:Wave):
        if sub_wave_is_extend(wave, 0):
            return -1
        return 0

class ImpluseWaveGuide5(Guide):
    desp = "如果浪3延长，则浪1和浪5的净位移（上涨的幅度）往往相同或者是斐波那契比率"
    def get_score(wave:Wave):
        if sub_wave_is_extend(wave, 2):
            return 0
        return wave_move_is_fibonacci(wave, 0, 4)

class ImpluseWaveGuide6(Guide):
    desp = "浪5或者浪1延长时，它往往与另外两浪的净位移呈斐波那契关系"
    def get_score(wave:Wave):
        score = 0
        if sub_wave_is_extend(wave, 0):
            score += wave_move_is_fibonacci(wave, 0, 4)
            score += wave_move_is_fibonacci(wave, 0, 2)
        if sub_wave_is_extend(wave, 4):
            score += wave_move_is_fibonacci(wave, 0, 4)
            score += wave_move_is_fibonacci(wave, 2, 4)
        return score
    
class ImpluseWaveGuide7(Guide):
    desp = "如果浪1是斜纹浪，浪3很可能会延长"
    def get_score(wave:Wave):
        if wave.sub_wave[0] and isinstance(wave.sub_wave[0], DiagonalWave):
            if sub_wave_is_extend(wave, 2):
                return 1
        return 0
    
class ImpluseWaveGuide8(Guide):
    desp = "如果浪3没有延长，浪5不太会是斜纹浪"
    def get_score(wave:Wave):
        # 三浪没有延长
        if not sub_wave_is_extend(wave, 2):
            if wave.sub_wave[4] and isinstance(wave.sub_wave[4], DiagonalWave):
                return -1
        return 0

class ImpluseWaveGuide9(Guide):
    desp = "浪2和浪4的形态交替, 浪2和浪4的调整一个陡直：会包含前一个浪的终点，另一个横向，不包含"

    def get_score(wave:Wave):
        def sub_wave_contains_start_point(wave, sub_wave_num=1):
            start_point = wave.point_list[sub_wave_num].price
            if not wave.sub_wave[sub_wave_num]:
                return False
            price_list = [ x.price for x in wave.sub_wave[sub_wave_num].point_list[1:]]
            if min(price_list) < previous_end and previous_end < max(price_list):
                return True
            return False
        wave_2_contains_start_point = sub_wave_contains_start_point(wave, 1)
        wave_4_contains_start_point = sub_wave_contains_start_point(wave, 3)
        if wave_2_contains_start_point != wave_4_contains_start_point:
            return 1
        return 0

class ImpluseWaveGuide10(Guide):
    desp = "浪2和浪4的形态交替, 浪2和浪4的调整过程形态往往不同，一个简单，一个复杂"

    def get_score(wave:Wave):
        if isinstance(wave.sub_wave[1], CombinationWave) and not isinstance(wave.sub_wave[3], CombinationWave):
            return 1
        if isinstance(wave.sub_wave[3], CombinationWave) and not isinstance(wave.sub_wave[1], CombinationWave):
            return 1   
        return 0
    
class ImpluseWaveGuide11(Guide):
    desp = "浪2常是一个锯齿形调整浪 Or 锯齿形联合调整浪"
    def get_score(wave:Wave):
        if isinstance(wave.sub_wave[1], ZigZagWave):
            return 1
        # 锯齿形联合调整浪
        if isinstance(wave.sub_wave[1], ZigZagCombinationWave):
            return 1
        return 0
        
class ImpluseWaveGuide12(Guide):
    desp = "浪4总是一个锯齿形调整浪 Or 三角形调整浪 Or 平台型联合调整浪"
    def get_score(wave:Wave):
        if isinstance(wave.sub_wave[3], ZigZagWave):
            return 1
        if isinstance(wave.sub_wave[3], TriangleWave):
            return 1
        if isinstance(wave.sub_wave[3], FlatCombinationWave):
            return 1
        return 0

class ImpluseWaveGuide13(Guide):
    desp = "当第五浪缩短时，第五浪未能超过第三浪，常出现在超强的第三浪后"
    def get_score(wave:Wave):
        if wave.get_sub_wave_move(2) > wave.get_sub_wave_move(0) * 2:
            # 第五浪未超过第三浪，即浪5的移动小于浪4
            if wave.get_sub_wave_move(4) < wave.get_sub_wave_move(3):
                return 1
        return 0
    
class ImpluseWaveGuide14(Guide):
    desp = "浪5预计较少的成交量"
    # TODO: implement me
    def get_score(wave:Wave):
        return 0
    
class CorrectiveWaveGuide1(Guide):
    desp = "第五浪延长后，调整浪会是陡直的形态，会在延长浪的浪2位置结束或者支撑。"
    # TODO: implement me
    def get_score(wave:Wave):
        return 0


In [ ]:
class MotiveWave(Wave):
    """
    驱动浪
    """
    is_extend_wave = False
    min_point_num = 6
    def __init__(self, point_list: list):
        super(MotiveWave, self).__init__(point_list)
        self.sub_wave = [None] * 5
    
class ImpluseWave(MotiveWave):
    """
    标准的推动5浪
    """
    rule_list = [Rule1, Rule2, Rule3, Rule4, Rule5, Rule6, Rule7, Rule8, Rule9]
    guide_dict = {
        ImpluseWaveGuide1: 1,
        ImpluseWaveGuide2: 1,
        ImpluseWaveGuide3: 1,
        ImpluseWaveGuide4: 1,
        ImpluseWaveGuide5: 1,
        ImpluseWaveGuide6: 1,
        ImpluseWaveGuide7: 1,
        ImpluseWaveGuide8: 1,
        ImpluseWaveGuide9: 1,
        ImpluseWaveGuide10: 1
    }

class DiagonalWave(MotiveWave):
    """
    斜纹浪
    """
    rule_list = []
    guide_dict = {
    }
    
class CorrectiveWave(Wave):
    """
    调整浪
    """
    pass

class ZigZagWave(CorrectiveWave):
    """
    锯齿形调整浪
    """
    pass

class FlatWave(CorrectiveWave):
    """
    平台型调整浪
    """
    pass

class TriangleWave(CorrectiveWave):
    """
    三角形调整浪
    """
    pass

class CombinationWave(CorrectiveWave):
    """
    联合型调整浪
    """
    pass

class ZigZagCombinationWave(CorrectiveWave):
    """
    锯齿形联合型调整浪
    """
    pass

class FlatCombinationWave(CorrectiveWave):
    """
    平台形联合型调整浪
    """
    pass

In [ ]:

def generate_wave(time_slot = 2000):
    # 1. 根据需要生成的时间段 * 20
    total_time_slot = time_slot * 20
    # 2. 生成第一个浪。生成第一个浪的过程需要先定义每个浪的位置和点位，先在范围内随机生成100个点，然后根据规则排除掉不符合规则的点，然后再根据指南进行分数排序，从最高分的10个选项中根据分数进行随机分布选取一种作为生成浪的位置。
    
    # 3. 在最大的父浪下，生成每个子浪。每个子浪先随机一个浪的类型，再根据2中的方式生成浪的点位和时间。
    # 4. 重复2、3直到子浪的范围足够小，例如只剩2、3、4个时间点时停止。
    # 5. 将生成的时间段的 K 线随机选取一个需要的时间段作为最终输出。
    

In [ ]:
import unittest

class TestRules(unittest.TestCase):
    def assert_valid_wave(self, wave):
        if not wave.is_valid():
            for rule in wave.get_not_valid_rule():
                print(f"Wave is not valid because: {rule.desp}")
        self.assertTrue(wave.is_valid())
        
    def assert_wave_score(self, wave, target):
        score_reason_dict = wave.get_score_contribution()
        for score_reason, score in score_reason_dict.items():
            print(score, score_reason)
        self.assertEqual(wave.get_score(), target)

    def test_rule1(self):
        point_list = [Point(1, 5), Point(10, 20), Point(20, 15), Point(40, 40), Point(50, 30), Point(60, 50)]
        wave = ImpluseWave(point_list)
        self.assert_valid_wave(wave)
        
        self.assert_wave_score(wave, 1)

        
        
unittest.main(argv=[''], verbosity=2, exit=False)
